In [29]:
from pydantic import BaseModel, ConfigDict
from typing import List
import random

In [33]:
class Point(BaseModel):
    coordinates: List[float]

class PythagoreanSupportMachineInput(BaseModel):
    points: List[Point]
    n_groups: int

class Group(BaseModel):
    points: List[Point]
    n_points: int
    centroid: Point

class PythagoreanSupportMachineOutput(BaseModel):
    groups: List[Group]
    n_dimensions: int

class Probability(BaseModel):
    probability: float
    point: Point

class Node(BaseModel):
    probabilities: List[Probability]
    centroid: Point

class ProbabilityMatrix(BaseModel):
    nodes: List[Node]

class Distance(BaseModel):
    distance: float
    point: Point
    group: Group

class Centroids(BaseModel):
    centroids: List[Point]    

In [16]:
def eject_object(base_model_obj, filename):
    with open(filename, 'w') as f:
        f.write(base_model_obj.model_dump_json())


In [36]:
def generate_points(
    n_dimensions: int,
    n_points: int,
    min_point_range: float,
    max_point_range: float
) -> List[Point]:
    points = []
    for _ in range(n_points):
        coords = [random.uniform(min_point_range, max_point_range) for _ in range(n_dimensions)]
        points.append(Point(coordinates=coords))
    return points


pts = generate_points(n_dimensions=2, n_points=5, min_point_range=-10, max_point_range=10)
input = PythagoreanSupportMachineInput(points=pts, n_groups=4)
eject_object(input, "input.json")

In [37]:
def get_first_probabilities(points: List[Point], n_groups: int) -> List[List[Probability]]:
    nodes: List[List[Probability]] = [[] for _ in range(n_groups)]
    
    for point in points:
        raw_probs = [random.random() for _ in range(n_groups)]
        total = sum(raw_probs)
        
        normalized_probs = [p / total for p in raw_probs]
        
        for i, prob in enumerate(normalized_probs):
            nodes[i].append(Probability(probability=prob, point=point.model_dump()))
    
    return nodes

def calculate_centroid_coordinate(probabilities: List[Probability], dimension: int) -> float:
    coordinate_numerator = 0
    coordinate_denominator = 0
    for probability in probabilities:
        coordinate_numerator += probability.point.coordinates[dimension] * (probability.probability) ** 2
        coordinate_denominator += (probability.probability) ** 2
    return coordinate_numerator / coordinate_denominator

def calculate_centroid(probabilities: List[Probability], n_dimensions: int) -> Point:
    coordinates = []
    for i in range(n_dimensions):
        coordinate = calculate_centroid_coordinate(probabilities, i)
        coordinates.append(coordinate)
    return Point(coordinates=coordinates)
    

def set_first_matrix(input: PythagoreanSupportMachineInput) -> ProbabilityMatrix:
    n_dimensions = len(input.points[0].coordinates)
    probabilities_list = get_first_probabilities(input.points, input.n_groups)
    nodes = []
    for probabilities in probabilities_list:
        centroid = calculate_centroid(probabilities, n_dimensions)
        nodes.append(Node(probabilities=probabilities, centroid=centroid))
    return ProbabilityMatrix(nodes=nodes)


result = set_first_matrix(input)
eject_object(result, "result.json")
